Необходимо понять какую метрику использовать чтобы оценивать качество моделей.

Я брал за основу следующие работы (5 наиболее цитируемых статей по запросу "object detection metrics" в google):

Performance Evaluation of Object Tracking Algorithms
====================================================

http://dircweb.kingston.ac.uk/papers/Yin,%20Fei2007_841079/PerformanceEvaluationofObjectTrackingAlgorithms.pdf 

Статья посвящена проблемы трекинга объектов на видео, а не на статическом изображении, поэтому для нас имеет вторичное значение. Сначала вводится понятие трека (размеченного бокса во времени), потом понятие пересечение боксов во времени ("истинного" и предсказанного системой). Таким образом все метрики представленные там предназначены для оценки треков.

Performance Evaluation of Object Detection and Tracking Systems
===============================================================

https://www.merl.com/publications/docs/TR2006-041.pdf

Большая статья посвященная проблеме трекинга объектов на видео. В отличие от предыдущей, авторы представили два набора метрик - первый так называемый "покадровый", как раз соотносится с нашей задачей (детектирования объектов на одном изображении), второй набор они называют "объектным" и он по сути соотносится с метриками треков из статьи описанной выше. В покадровых метриках фактически вводится понятия True Negative, False Negative, и т.д. на каждом кадре. Причем считается что детектирование верное, если "истинный" бокс и предсказанный хоть как-то пересекаются. Для нашей задачи это выглядит через чур простой метрикой.


Performance Detection and Object Tracking in Video
==================================================

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.74.6602&rep=rep1&type=pdf

Ещё одна статья не по той теме. Параграф 3.1 посвящен покадровым метрикам. Вводится понятие FDA - Frame Detection Accuracy.

$ FDA = \cfrac {Overlap \space ratio} {\cfrac {N_g + N_d} {2} } $

где $N_g$ - количество "истинных" объектов на изображении, $N_d$ - количество объектов на изображении, отмеченных системой.

$Overlap \space ratio = \displaystyle\sum_{i=1}^{N_{mapped}} \cfrac {N_g^i \cap N_d^i} {N_g^i \cup N_d^i} $ или проще говоря сумма мер степени "пересечения" заданного"истинного" и отмеченного системой бокса. Для нашей задачи надо такую метрику считать для каждого класса (текст, фигура, доска) отдельно и потом суммировать результат.
Остальные метрики в статье строятся без учета точности детектирования, которая нас интересует.

Performance Evaluation of Object Detection Algorithms
=====================================================

https://www.researchgate.net/publication/220928471_Performance_Evaluation_of_Object_Detection_Algorithms

Снова статья про детектирование объектов на видео. Все метрики сначала объявляются для отдельного кадра, затем обобщаются на последовательность кадров. В отличие от предыдущих статей, тут много подходящих для нашей задачи метрик. Для каждой метрики описаны её достоинства и недостатки. Опишу покадровые метрики.

Area-Based Recall for Frame
---------------------------

Это площадная (попиксельная) метрика т.е. она не оперирует боксами - равна отношению правильно размеченной площади к общей "истинной" площади. Понятно что она предрасположена к большим объектам (т.к. её значение например будет максимально в вырожденном случае когда всё изображение размечено как "объект". В силу этого недостатка брать её мы не будем.

Area-based Precision for Frame
------------------------------
Площадная метрика - минимизирует количество неверно размеченной площади. Равна отношению правильно размеченной площади к общему количеству размеченной (алгоритмом) площади. Кажется что эта метрика подходит для нашей задачи.

Average Fragmentation
---------------------
Данная метрика направлена на минимизацию фрагментации детектирования чтобы уменьшить вероятность того что детектор разрежет один объект на много кусочков. Это сделано с учетом нашего случая - когда детектированный кусок затем подается алгоритму распознавания. Таким образом эта метрика представляет для нас интерес.
Интересно, что метрика штрафует как за не пересекающиеся, так и за пересекающиеся боксы. Возможно, в нашем случае будет правильно не штрафовать за пересекающиеся боксы т.к. наши объекты имеют сложную форму и их часто просто невозможно описать одним боксом. 
Метрика определяется для каждого объекта на изображении отдельно. Это усложняет дело т.к. в нашей разметке один объект часто размечен несколькими боксами, необходимо будет провести переразметку чтобы распределить боксы по группам. Метрика равна $ \frac 1 {1 + \log N} $, где N - количество боксов отмеченных для данного объекта, которые пересекаются с истинной разметкой данного объекта.
Метрика не учитывает площади пересечения, что является большим недостатком.

Average Object Area Recall
--------------------------

Определяется сначала для отдельного объекта - равна отношению площади объекта верной размеченной алгоритмом к общей площали объекта. Значение метрики для всего кадра равно среднему арифметическому от значений для каждого объекта.
Метрика учитывает площадь пересечения, но не учитывает площади самих объектов: значение будет равно 0.5 в случае если был правильно задетекчен огромный объект, но пропущен второй размером в 1 пиксель.

Average Detected Box Area Precision
-----------------------------------

Метрика вычисляется не для объектов, а для боксов - что хорошо подходит для нашей текущей разметки. Для каждого бокса отмеченного алгоритмом вычисляется его precision - пропорция его площади, правильно задетекченная алгоритмом. Затем берется среднее арифметическое этих значений по всем боксам которые расставил алгоритм. Все боксы вносят вклад в метрику не зависящий от их размера. Таким образом метрику нельзя обмануть очень маленькими или очень большими боксами. 

Localized Object Count Recall
-----------------------------
Данная метрика считает объект задетектированным, если некоторая пропорция его площади была правильно покрыта боксами. В остальном это тоже самое что и Average Object Area Recall, только считается количество задетекченных объектов, вместо площади. 

Localized Output Box Count Precision
------------------------------------
Аналог Localized Object Count Recall, но за базу взята метрика Average Detected Box Area Precision. Недостатком упомянуто то, что метрика не штрафует за обозначение одним боксом нескольких "истинных" объектов, т.е. за неправильную сегментацию.

Performance Measures for Object Detection Evaluation
====================================================

http://www.cs.umd.edu/~ozdemir/papers/prl_evaluation.pdf

Статья содержит описание продвинутого алгоритма вычисления меры который разработали авторы базируясь на контесте по детектированию зданий на спутниковых фотографиях. Алгоритм работает в три стадии:
* находится соответствие между разметкой алгоритма и "истинными" объектами
* измерение каждого найденного на первом шаге соответствия
* Финальное ранжирование результатов второй стадии

В качестве плюсов данного алгоритма упомянуто что:
1) Он умеет обрабатывать соответствие one-to-many, many-to-one, many-to-many в отличие от остальных метрик, которые работают на уровне one-to-one.
2) Метрика каждого соответствия базируется не только на площади, но и на других геометрических параметрах

Предлагается использовать данный алгоритм в место Precision/Recall метрик из предыдущей статьи в случае, когда необходимо точно (в смысле геометрических границ) детектить множество объектов на одном изображении.

Дан краткий обзор работ по данной тематике которые уделяют внимание вопросу аккуратности детектирования границ объектов.
По пиксельным (площадным) метрикам:
https://pdfs.semanticscholar.org/8e04/3a5e3a89f095e80c78a98d34c42654243c95.pdf  
http://www.gipsa-lab.grenoble-inp.fr/~jocelyn.chanussot/publis/jei_08_lemeur_roc.pdf  

(и много других ссылок см. в статье)

Далее обсуждается первая стадия (матчинг задетекченных объектов с размеченными). Предлагается три алгоритма:
первый работает с двудольным графом, где слева - задетекченные боксы - слева - размеченные объекты. Вес ребра - пересечение по площади. Предлагают искать двудольный подграф с максимальным весом ребер с помощью венгерского алгоритма (http://e-maxx.ru/algo/assignment_hungary). Такой способ детектит только соответствия one-to-one

Второй вычисляет специальную метрику: Hoover index, которая имеет один свободный параметр (порог) T и позволяет детектить любые типы соответствий.

Третий способ заключается в том чтобы использовать нелинейное численное программирование.

Далее обсуждаются метрики применяемые для каждого совпадения. В дополнении к двум метрикам из предыдущей статьи предлагается использовать приличное количество матана (тервер + решение оптимизационной задачи + специальную меру (в геометрическом смысле) - Mallows distance)

Далее обсуждается ранжирование, предлагается для него использовать precision/recall/accuracy метрики.

Далее обсуждается алгоритмическая сложность всего этого дела, которая оказывается экспоненциальной :D

В целом это слишком сложный фреймворк чтобы с него начинать, но тут есть потенциально полезные ссылки на статьи по правильному определению границ детектированных объектов. 



Выбор метрики
=============

Я выбираю метрику Localized Output Box Count Precision из всех тех что были описаны выше. В качестве порога выберем "разумное с потолка" соотношение 0.8 верно задетекченной площади к общей площади объекта.
Мною руководят следующие соображения:

* На данном этапе нам не так важна точность сегментации
* Метрика не чувствительна к большим/маленьким боксам
* Выбор порога вынуждает алгоритм к поиску довольно точных покрытий объектов боксами

Следует пересмотреть выбор метрики в случае, если данные факторы изменятся. Также можно провести доразметку датасета с выделением отдельных объектов, чтобы выбрать другие метрики (recall based).

Даю формальное описание метрики:

In [3]:
%%latex

\begin{equation}
 LOBCP = 
 \displaystyle\sum_{i=1}^{N} 
    \begin{cases}
      1, & \text{if}\ \cfrac {| D_i \cap U_g |} {|D_i|} > Overlap_{min} \\
      0, & \text{otherwise}
    \end{cases}
  \end{equation}

<IPython.core.display.Latex object>

где $N$ - количество боксов выданных алгоритмом, $|D_i|$ - площадь i-го бокса выданного алгоритмом, $| D_i \cap U_g |$ - площадь пересечения бокса с "истинной" разметкой и  $ Overlap_{min} = 0.8 $ 

Метрику будем вычислять отдельно для каждого класса (фигура, текст) и потом суммировать результат.

Для доски мы знаем что у нее 4 угла которые алгоритму необходимо точно отметить, поэтому логично сразу минимизировать MSE отклонения углов детекченных алгоритмом от размеченных углов доски. При этом может быть так что доски вообще на картинке нет, поэтому нужно отдельно штрафовать за непонимание этого.

$ M_{whiteboard} = \frac 1 4 \displaystyle\sum_{i=1}^{4} (X_{i, U} - X_{i, G})^2  + (Y_{i, U} - Y_{i, G})^2 $
